In [1]:
import os
import glob
import time
import torch
import random
import logging
import argparse

import numpy as np 
import pandas as pd 
import pytorch_lightning as pl

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

from transformers import (
    AdamW,
    AutoConfig,
    T5ForConditionalGeneration,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
)

In [2]:
train = pd.read_csv('tweet-sentiment-extraction/train.csv').dropna()
test = pd.read_csv('tweet-sentiment-extraction/test.csv')

train, val = train_test_split(train, test_size=0.13, random_state=42)

In [3]:
train.head(), test.head(), val.head()

(           textID                                               text  \
 17391  a4ccd2c1a9   How did we just get paid and still be broke a...   
 10951  5c3d9c52f6  i no i no bt i had only been a gamer for like ...   
 19109  05bc7a77ef  I love when my ipod shuffles so all the good s...   
 16873  cfb49dff56   no i mean 2moz. I`m workin` 7-1 in a bakers t...   
 22461  f497495e23  Lovely walk this morning with the missus; driz...   
 
                                            selected_text sentiment  
 17391                                    broke as hell?!  negative  
 10951                                               luvd  positive  
 19109                                               love  positive  
 16873  no i mean 2moz. I`m workin` 7-1 in a bakers th...   neutral  
 22461  Lovely walk this morning with the missus; driz...  positive  ,
        textID                                               text sentiment
 0  f87dea47db  Last session of the day  http://twitpic.com/67e

In [4]:
# Input
for a,b,_ in zip(train.sentiment.values[:10], train.text.values[:10], train.selected_text.values[:10]):
    print("sentiment:", a, "tweet:", b)

sentiment: negative tweet:  How did we just get paid and still be broke as hell?! No shopping spree for me today
sentiment: positive tweet: i no i no bt i had only been a gamer for like 2 years when i made that attempt  lol yea i luvd F1 to an extent 
sentiment: positive tweet: I love when my ipod shuffles so all the good songs are all together
sentiment: neutral tweet:  no i mean 2moz. I`m workin` 7-1 in a bakers then 6-4 later in a pub
sentiment: positive tweet: Lovely walk this morning with the missus; drizzle didn`t matter
sentiment: neutral tweet:  , just dont understand what`s it got to do with me. I`m just a nice girl
sentiment: negative tweet: getting bored of walking up and down the stairs
sentiment: positive tweet:  have your own style. it just might work.
sentiment: negative tweet: fighting with mum on mothers day
sentiment: neutral tweet:  & I got too much work to do


In [5]:
# Target (what we're trying to predict)
for _,_,c in zip(train.sentiment.values[:10], train.text.values[:10], train.selected_text.values[:10]):
    print(c)

broke as hell?!
luvd
love
no i mean 2moz. I`m workin` 7-1 in a bakers then 6-4 later in a pub
Lovely walk this morning with the missus; drizzle didn`t matter
, just dont understand what`s it got to do with me. I`m just a nice girl
getting bored of walking up and down the stairs
it just might work.
fighting
I got too much work to do
